In [ ]:
#!/usr/bin/env python3

import io
import os
import sys
import time
import picamera
import logging
import webbrowser 
import numpy as np
import smtplib, ssl
from numpy import asarray
import socketserver
from time import sleep
from threading import Condition
from http import server
from PIL import Image
from urllib.parse import parse_qs

        
## Create Settings Manager  (Must instantiate too intialize from file first)
userSettings = settingManager()

## Create Motion Detector
moDet = motionDetector()   
# Call is: 
# moDet.processFrame(frame)

## Create Network Core
neCo = netCore()
# Call is: 
# result1 = neCo.turingTest(sample)
# result2 = neCo.indentifySubject(sample)

## Create Notification Manager
noteMan = notificationManager()

## Begin Streaming
PAGE = userSettings.getPage()
STYLESHEET = userSettings.getStyles()
# JAVASCRIPT =  userSettings.getJava()

class StreamingOutput(object):

    def __init__(self):
        self.frame = None
        self.buffer = io.BytesIO()
        self.condition = Condition()

    def write(self, buf):
        if buf.startswith(b'\xff\xd8'):
            # New frame, copy the existing buffer's content and notify all
            # clients it's available
            self.buffer.truncate()
            with self.condition:
                self.frame = self.buffer.getvalue()
                self.condition.notify_all()
                moDet.processFrame(self.frame)

#                 camera.annotate_text_size = 6
#                 camera.annotate_foreground = picamera.Color('white') # white is also the default
#                 camera.annotate_foreground = picamera.Color(y=0.4, u=0, v=0) # dark
#                 camera.annotate_foreground = picamera.Color(y=0.2, u=0, v=0) # darker
#                 camera.annotate_foreground = picamera.Color(y=0.0, u=0, v=0) # average?
#                 camera.annotate_foreground = picamera.Color('black') # black
#                 camera.annotate_text = ("Captured "+str(moDet.imgNum)+" images")
#                 camera.annotate_text = str(moDet.imgNum)
                self.buffer.seek(0)

        return self.buffer.write(buf)

    
    
class StreamingHandler(server.BaseHTTPRequestHandler):
    
    def _set_headers(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        self.shutdownFlag = 'R'
        
    def setShutdownFlag(passedValue):
        self.shutdownFlag = passedValue
        return
    
    def getShutdownFlag():
        return self.shutdownFlag
    
    def do_POST(self):
        print(self.headers)

        content_length = int(self.headers.get('Content-Length', 0))
        content_string = self.rfile.read(content_length).decode("UTF-8")
        fields = parse_qs(content_string)
        handleInput(fields)
        
        self.send_response(302)
        self.send_header('Location', '/index.html')
        self.end_headers()
        self._set_headers()   
        return
    
    def do_GET(self):
        if self.path == '/':
            self.send_response(301)
            self.send_header('Location', '/index.html')
            self.end_headers()
        elif self.path == '/index.html':
            content = PAGE.encode('utf-8')
            self.send_response(200)
            self.send_header('Content-Type', 'text/html')
            self.send_header('Content-Length', len(content))
            self.end_headers()
            self.wfile.write(content)
        elif self.path == '/style.css':
            content = STYLESHEET.encode('utf-8')
            self.send_response(200)
            self.send_header('Content-Type', 'text/css')
            self.send_header('Content-Length', len(content))
            self.end_headers()
            self.wfile.write(content)
#         elif self.path == '/javascript.js':
#             content = JAVASCRIPT.encode('utf-8')
#             self.send_response(200)
#             self.send_header('Content-Type', 'text/javascript')
#             self.send_header('Content-Length', len(content))
#             self.end_headers()
#             self.wfile.write(content)
        elif self.path == '/stream.mjpg':
            self.send_response(200)
            self.send_header('Age', 0)
            self.send_header('Cache-Control', 'no-cache, private')
            self.send_header('Pragma', 'no-cache')
            self.send_header('Content-Type', 'multipart/x-mixed-replace; boundary=FRAME')
            self.end_headers()

            try:
                while True:
                    with output.condition:
                        output.condition.wait()
                        frame = output.frame
                        
                        self.wfile.write(b'--FRAME\r\n')
                        self.send_header('Content-Type', 'image/jpeg')
                        self.send_header('Content-Length', len(frame))
                        self.end_headers()
                        self.wfile.write(frame)
                        self.wfile.write(b'\r\n')

            except Exception as e:
                if self.shutdownFlag=="x":
                    camera.close()
                    server.shutdown()
                    server.server_close()
                    print("Remote Shutdown Requested.")
                logging.warning(
                    'Removed streaming client %s: %s',
                    self.client_address, str(e))
        else:
            self.send_error(404)
            self.end_headers()

            
            
class StreamingServer(socketserver.ThreadingMixIn, server.HTTPServer):
    allow_reuse_address = True
    daemon_threads = True

with picamera.PiCamera(resolution='240x240', framerate=24) as camera: #'729x729', framerate=6) as camera:
    camera.rotation = 270
    output = StreamingOutput()
    camera.start_recording(output, format='mjpeg')
#    webbrowser.open('http://10.0.0.201:7777', new=0, autoraise=True)
        
    try:
        address = ('', 7777)
        server = StreamingServer(address, StreamingHandler)
        print("Starting Server . . .")
        if notificationType==0:
            print("Notifications are disabled.")
        elif notificationType==1:
            print("Notifications are set to Motion.")
        elif notificationType==2:
            print("Notifications are set to Turing.")
        server.serve_forever() #poll_interval=0.5)
        
    except KeyboardInterrupt:
        camera.close()
        server.shutdown() # Stop the serve_forever
        server.server_close()  # Close also the socket.
        print("Shutdown requested")
        
    except Exception:
        print("Encountered Unhandled Error!")
        camera.close()
        server.shutdown() # Stop the serve_forever
        server.server_close()  # Close also the socket.

    finally:
        camera.close()
        server.shutdown() # Stop the serve_forever
        server.server_close()  # Close also the socket.